In [1]:
import json
import datetime
from collections import defaultdict

dayOfWeekReviews = defaultdict(dict)
with open('./data/yelp_academic_dataset_review.json', 'rb') as fread:
    for line in fread:
        jsonData = json.loads(line)
        date = datetime.datetime.strptime(jsonData['date'], '%Y-%m-%d')
        star = jsonData['stars']
        weekDay = date.weekday()
        
        weekDayStars = dayOfWeekReviews[weekDay]
        if star in weekDayStars:
            weekDayStars[star] += 1
        else:
            weekDayStars[star] = 1
            
        dayOfWeekReviews[weekDay] = weekDayStars

for k, v in dayOfWeekReviews.iteritems():
    print k, v


0 {1: 24514, 2: 22501, 3: 36661, 4: 75243, 5: 88103}
1 {1: 22734, 2: 20642, 3: 33634, 4: 70039, 5: 85669}
2 {1: 22282, 2: 20231, 3: 32798, 4: 69135, 5: 87414}
3 {1: 20965, 2: 18322, 3: 29390, 4: 62863, 5: 80326}
4 {1: 21701, 2: 18075, 3: 28055, 4: 59687, 5: 77408}
5 {1: 23251, 2: 19461, 3: 28929, 4: 60933, 5: 79878}
6 {1: 24364, 2: 21376, 3: 33252, 4: 68699, 5: 80729}


In [2]:
for i in xrange(7):
    d = dayOfWeekReviews[i]
    print d[1], d[2], d[3], d[4], d[5], i

24514 22501 36661 75243 88103 0
22734 20642 33634 70039 85669 1
22282 20231 32798 69135 87414 2
20965 18322 29390 62863 80326 3
21701 18075 28055 59687 77408 4
23251 19461 28929 60933 79878 5
24364 21376 33252 68699 80729 6


In [3]:
import json
import datetime
from collections import defaultdict

businessTypes = defaultdict(int)
businessCountByCity = defaultdict(int)
businessTypeCountByCity = defaultdict(lambda : defaultdict(int))

with open('./data/yelp_academic_dataset_business.json', 'rb') as fread:
    for line in fread:
        jsonData = json.loads(line)
        city = jsonData['city']
        state = jsonData['state']
        categories = jsonData['categories']
        for cat in categories:
            businessTypes[cat] += 1
            businessTypeCountByCity[(city, state)][cat] += 1
            
        businessCountByCity[(city, state)] += 1
        

In [10]:
print len(businessCountByCity)

391


In [5]:
#for k, v in businessCountByCity.iteritems():
#    print k[0], '|', k[1], '|', v
import operator

print max(businessCountByCity.iteritems(), key=operator.itemgetter(1))[1]

13600


In [8]:
value = 1
dailyIncrease = .18/100
for i in xrange(36):
    value += value * dailyIncrease

print value

1.92785774278


In [8]:
reviewLengthByStar = defaultdict(list)
numOfStarsByUid = defaultdict(lambda : defaultdict(int))
with open('./data/yelp_academic_dataset_review.json', 'rb') as fread:
    for line in fread:
        jsonData = json.loads(line)
        star = jsonData['stars']
        weekDay = date.weekday()
        text = jsonData['text']
        userId = jsonData['user_id']
        reviewLengthByStar[star].append(len(text))
        numOfStarsByUid[userId][star] += 1


In [13]:
import numpy as np
for star in reviewLengthByStar:
    reviewLengths = reviewLengthByStar[star]
    print star, np.mean(reviewLengths), np.std(reviewLengths), len(reviewLengths)

1 808.677519069 754.153286099 159811
2 822.279656919 700.89973176 140608
3 753.1022589 637.060557939 222719
4 674.873238048 591.743440854 466599
5 574.621950315 553.945944464 579527


In [ ]:
for cat, count in businessTypes.iteritems():
    print cat, '|', count

In [1]:
import toyplot.color
toyplot.color.Palette()

In [ ]:
import numpy as np

np.random.seed(1234)
observations = np.random.normal(size=(50, 50))

x = np.linspace(0, 1, len(observations))
y1 = np.min(observations, axis=1)
y2 = np.max(observations, axis=1)

canvas = toyplot.Canvas(width=400, height=300)
axes = canvas.axes()
mark = axes.fill(x, y1, y2)